In [67]:
import bonobo
from requests_html import HTMLSession
from pprint import pprint
import dotenv
import os
import json
import pandas as pd
from geopy import Nominatim
from geopy.distance import geodesic
import numpy as np

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

dotenv.load_dotenv()

True

In [68]:
np.round(0.0123124, 2)

0.01

In [2]:
geocoder = Nominatim(user_agent='Kiga Graz')

In [3]:
mapbox_access_token = os.getenv('MAPBOX')

In [4]:
with open('aggregate/data/krippe_stad.json') as f:
    krippe_stad = json.load(f)
with open('aggregate/data/krippe_privat.json') as f:
    krippe_privat = json.load(f)
with open('aggregate/data/landmarks.json') as f:
    landmarks = json.load(f)

In [5]:
df_stad = pd.DataFrame(krippe_stad)
df_privat = pd.DataFrame(krippe_privat)
df_landmarks = pd.DataFrame(landmarks)

In [8]:
home = geocoder.geocode("Grabenstrasse 45, 8010 Graz")
home

Location(45, Grabenstraße, Geidorf, Graz, Steiermark, 8010, Österreich, (47.0844163, 15.4366836263287, 0.0))

In [64]:
df = pd.concat([df_stad, df_privat], sort=False)
df = df[['name', 'address', 'lat', 'lon']].dropna()
df['distance'] = df.apply(lambda x: geodesic((x.lat, x.lon), (home.latitude, home.longitude)).km, axis=1)
df.head()

,name,address,lat,lon,distance
0,Kinderkrippe Plüddemanngasse,"Plüddemanngasse 28, 8010 Graz",47.066873,15.462077,2.742869
1,Kinderkrippe Erlengasse,"Erlengasse 25, 8020 Graz",47.089034,15.415219,1.708807
2,Kinderkrippe Idlhofgasse,"Idlhofgasse 74, 8020 Graz",47.064953,15.422393,2.420778
3,Kinderkrippe Dornschneidergasse,"Dornschneidergasse 45, 8020 Graz",47.049065,15.430726,3.956020
4,Kinderkrippe Petersgasse,"Petersgasse 94, 8010 Graz",47.060695,15.460916,3.215879


In [66]:
df.sort_values(by='distance', inplace=True)
df.head()

,name,address,lat,lon,distance
5,Kinderkrippe Landesunterstützungsverein,"Körblergasse 4, 8010 Graz",47.078871,15.444067,0.833321
10,Kinderkrippe integrativ geführt für Hör- und S...,"Rosenberggürtel 12, 8010 Graz",47.081173,15.448589,0.973348
7,ÖH Uni Graz/ WIKI Kinderkrippe,"Rosenberggürtel 12, 8010 Graz",47.081173,15.448589,0.973348
8,Kinderkrippe Max und Moritz,"Glacisstraße 9, 8010 Graz",47.076737,15.444657,1.046671
13,Anton-Afritsch Kinderkrippe,"Gabelsbergerstraße 22, 8020 Graz",47.077269,15.426316,1.118603


In [11]:
data = [
    go.Scattermapbox(
        name='Stadt',
        lat=df_stad.lat,
        lon=df_stad.lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=df_stad.GT*5,
            color='green'
        ),
        text=df_stad.name,
    ), 
    
    go.Scattermapbox(
        name='Privat',
        lat=df_privat.lat,
        lon=df_privat.lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color='blue'
        ),
        text=df_privat.name,
    ),
    
    go.Scattermapbox(
        name='Landmarks',
        lat=df_landmarks.lat,
        lon=df_landmarks.lon,
        mode='markers+text',
        textposition="bottom center",
        marker=go.scattermapbox.Marker(
            size=10,
            color='red'
        ),
        text=df_landmarks.name,
        textfont=dict(color='red')
    )
]



layout = go.Layout(
    title='Kinderkrippe Graz 2019',
    autosize=True,
    width=1024,
    height=1024,
    legend=dict(orientation="h", x=0, y=0),
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=df_landmarks.loc[0].lat,
            lon=df_landmarks.loc[0].lon
        ),
        pitch=0,
        zoom=12
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='Kiderkrippe Graz')